# The Simplex algotithm

A simple implementation for the primal formulation of the simplex algorithm.

## Formulation of a simple problem

In [1]:
import numpy as np
from fractions import Fraction

In [2]:
_A = np.array(
    [
        [1, 3, 1, 1, 0, 0, 0],
        [-1, 0, 3, 0, 1, 0, 0],
        [2, -1, 2, 0, 0, 1, 0],
        [2, 3, -1, 0, 0, 0, 1]
    ],
    dtype=np.int64
)

_A

array([[ 1,  3,  1,  1,  0,  0,  0],
       [-1,  0,  3,  0,  1,  0,  0],
       [ 2, -1,  2,  0,  0,  1,  0],
       [ 2,  3, -1,  0,  0,  0,  1]])

In [3]:
_p = np.array([3, 2, 4, 2], dtype=np.int64)
_p

array([3, 2, 4, 2])

In [4]:
_B = np.hstack([_A, _p[..., np.newaxis]])
_B

array([[ 1,  3,  1,  1,  0,  0,  0,  3],
       [-1,  0,  3,  0,  1,  0,  0,  2],
       [ 2, -1,  2,  0,  0,  1,  0,  4],
       [ 2,  3, -1,  0,  0,  0,  1,  2]])

In [5]:
_z = np.array([5, 5, 3, 0, 0, 0, 0] + [0], dtype=np.int64)
_z

array([5, 5, 3, 0, 0, 0, 0, 0])

In [6]:
from mcf_simplex_analyzer.load_instance import FractionArray

In [7]:
A = FractionArray(_A, np.ones_like(_A))
A

FractionArray(numerator=array([[ 1,  3,  1,  1,  0,  0,  0],
       [-1,  0,  3,  0,  1,  0,  0],
       [ 2, -1,  2,  0,  0,  1,  0],
       [ 2,  3, -1,  0,  0,  0,  1]]), denominator=array([[1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1]]))

In [8]:
p = FractionArray(_p, np.ones_like(_p))
p

FractionArray(numerator=array([3, 2, 4, 2]), denominator=array([1, 1, 1, 1]))

In [9]:
B = FractionArray(_B, np.ones_like(_B))
B

FractionArray(numerator=array([[ 1,  3,  1,  1,  0,  0,  0,  3],
       [-1,  0,  3,  0,  1,  0,  0,  2],
       [ 2, -1,  2,  0,  0,  1,  0,  4],
       [ 2,  3, -1,  0,  0,  0,  1,  2]]), denominator=array([[1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1]]))

In [10]:
base_ind = np.array([3, 4, 5, 6])
base = np.zeros_like(_B[0], dtype=bool)
base[base_ind] = True
base

array([False, False, False,  True,  True,  True,  True, False])

In [11]:
# TODO: How to determine this?
row_to_var_index = np.array([3, 4, 5, 6])
var_index_to_row = np.array([0, 0, 0, 0, 1, 2, 3])

In [12]:
z = FractionArray(_z, np.ones_like(_z))
z

FractionArray(numerator=array([5, 5, 3, 0, 0, 0, 0, 0]), denominator=array([1, 1, 1, 1, 1, 1, 1, 1]))

## Entering variable (Dantzig)

In [13]:
def dantzig_entering(z, base):
    positive = np.where(~base)[0]
    print(z.numerators[positive], z.denominators[positive])
    lcm_z = np.lcm.reduce(z.denominators[positive])
    print((lcm_z // z.denominators[~base]) * z.numerators[positive])
    entering = positive[np.argmax((lcm_z // z.denominators[positive]) * z.numerators[positive])]
    
    return entering

entering = dantzig_entering(z, base)
entering

AttributeError: 'FractionArray' object has no attribute 'numerators'

In [ ]:
# Possible speedup: Convert to floats
rng = np.random.default_rng()
x = rng.uniform(size=1000)
m = x.max()
eps = 0.001
np.where((m - eps <= x) & (x <= m + eps))[0]

## Leaving variable (Dantzig)

In [ ]:
B.denominators[..., -1] * B.numerators[..., entering]
B.denominators[..., -1]
B.numerators[..., entering]

In [ ]:
def dantzig_leaving(B, entering):
    lcm_B = np.lcm.reduce(B.denominators[..., entering])
    np.amin(B.numerators[..., entering] * lcm_B)
    
    positive = np.where(B.numerators[..., entering] > 0)[0]
    print(positive)
    
    bound_nominators = B.numerators[..., -1][positive] * B.denominators[..., entering][positive]
    bound_denominators = B.denominators[..., -1][positive] * B.numerators[..., entering][positive]

    print(bound_denominators, bound_nominators)
    lcm_bound = np.lcm.reduce(bound_denominators)
    bounds = bound_nominators * (lcm_bound // bound_denominators)
    
    valid = np.where(bounds >= 0)[0]
    
    leaving_row = positive[valid[bounds[valid].argmin()]]
    
    return leaving_row

In [ ]:
leaving_row = dantzig_leaving(B, entering)
leaving_row

In [ ]:
leaving = row_to_var_index[leaving_row]
leaving

## Update

In [ ]:
def determine_update_row(B, leaving_row):
    new_row_num = B.numerators[leaving_row] * B.denominators[leaving_row][entering]
    new_row_denom = B.denominators[leaving_row] * B.numerators[leaving_row][entering]
    
    new_row_gcd = np.gcd(new_row_num, new_row_denom)
    update_row = FractionArray(new_row_num // new_row_gcd, new_row_denom // new_row_gcd)
    
    return update_row

In [ ]:
update_row = determine_update_row(B, leaving_row)

In [ ]:
B

In [ ]:
def update_table(B, entering, leaving_row):
    update_row = determine_update_row(B, leaving_row)
    
    B.numerators[leaving_row] = update_row.numerators
    B.denominators[leaving_row] = update_row.denominators
    
    for row in range(B.numerators.shape[0]):
        if row == leaving_row:
            continue

        coeff =  B[row, entering]
        
        update_num = - coeff.numerator * update_row.numerators
        update_denom = coeff.denominator * update_row.denominators
        
        print(coeff)
        print(update_row.numerators)
        print(update_row.denominators)
        
        print(update_num, update_denom)

        new_num = update_num * B.denominators[row] + B.numerators[row] * update_denom
        new_denom = update_denom * B.denominators[row]

        print(new_num, new_denom)
        update_gcd = np.gcd(new_num, new_denom)

        print(update_gcd)
        print(new_num // update_gcd, new_denom // update_gcd)
        print()

        B.numerators[row] = new_num // update_gcd
        B.denominators[row] = new_denom // update_gcd

In [ ]:
update_table(B, entering, leaving_row)
B

In [ ]:
def update_objective(z, entering, leaving_row):
    coeff = z[entering]

    update_num = coeff.numerator * update_row.numerators
    update_num[:-1] *= -1
    update_denom = coeff.denominator * update_row.denominators

    new_num = update_num * z.denominators + z.numerators * update_denom
    new_denom = update_denom * z.denominators

    update_gcd = np.gcd(new_num, new_denom)

    z.numerators = new_num // update_gcd
    z.denominators  = new_denom // update_gcd

In [ ]:
update_objective(z, entering, leaving_row)
z

In [ ]:
def update_base(base, entering, leaving_row):
    row_to_var_index[leaving_row] = entering
    var_index_to_row[entering] = var_index_to_row[leaving]

    base[entering] = True
    base[leaving] = False
    
    return base
    
update_base(base, entering, leaving_row)
base

# Second iteration

In [ ]:
entering2 = dantzig_entering(z, base)
entering2

In [ ]:
leaving_row2 = dantzig_leaving(B, entering2)
leaving_row2

In [ ]:
leaving2 = row_to_var_index[leaving_row2]
leaving2

In [ ]:
update_table(B, entering2, leaving_row2)
update_objective(z, entering2, leaving_row2)
update_base(base, entering2, leaving_row2)
B, z, base